In [7]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import math 
import copy
import random
import argparse
import datetime
print(datetime.datetime.now().strftime("%d%B%Y, %H:%M:%S"))
device = torch.device('cpu')

# data loader
# 返回对象维context_size, arm_size, context, reward, sli(排序)
def load_data(dataset_name, K):
    if dataset_name == "statlog":
        ##statlog
        __context_size__ = 8
        __arm_size__ = 7
        fr = open("shuttle.trn",'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K+=1
            aaa = line.split("\n")
            aaa = aaa[0].split(" ")
            temp = np.int64(aaa[9])-1
            context = np.double(aaa[1:9])
            context = context/np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)

    TEMP_CONTEXT = []
    TEMP_REWARD = []
    sli = np.random.permutation(K)
    for i in range(K):
        TEMP_CONTEXT.append(CONTEXT[sli[i]])
        TEMP_REWARD.append(REWARD[sli[i]])
    CONTEXT = TEMP_CONTEXT
    REWARD = TEMP_REWARD

    return __context_size__, __arm_size__, CONTEXT, REWARD, sli

09October2023, 18:26:29


In [8]:
a,b,context, reward, sli = load_data("statlog", 15000)

In [14]:
context

[array([0.        , 0.76060165, 0.        , 0.43216003, 0.08643201,
        0.33708482, 0.33708482, 0.        ]),
 array([ 0.        ,  0.76168064, -0.02821039,  0.39494552, -0.08463118,
         0.34792819,  0.36673512,  0.01880693]),
 array([ 0.        ,  0.66114754, -0.04450032,  0.10171501, -0.17164407,
         0.42593159,  0.56578972,  0.13985813]),
 array([0.00934498, 0.75694323, 0.        , 0.35510917, 0.        ,
        0.38314411, 0.39248908, 0.        ]),
 array([0.        , 0.75789312, 0.03525084, 0.38775927, 0.        ,
        0.37013385, 0.37013385, 0.        ]),
 array([0.        , 0.7694452 , 0.08099423, 0.46571683, 0.        ,
        0.30372837, 0.30372837, 0.        ]),
 array([ 0.        ,  0.72871721,  0.        ,  0.24929799, -0.14382577,
         0.38353538,  0.47941922,  0.09588384]),
 array([ 0.        ,  0.70107271,  0.        ,  0.16388713, -0.16388713,
         0.37329846,  0.53718558,  0.16388713]),
 array([ 0.        ,  0.7700374 ,  0.06079243,  0.445811

In [15]:
def TRANS(c, a, arm_size):
    #### transfer an array context + action to new context with dimension 2*(__context__ + __armsize__)
    dim = len(c)
    action = np.zeros(arm_size)
    action[a] = 1
    c_final = np.append(c, action)
    c_final = torch.from_numpy(c_final).to(device)
    c_final = c_final.view((len(c_final), 1))
    c_final = c_final.repeat(2, 1)
    c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)
    return c_final

In [16]:
reward

[array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0.]),
 array([0., 

In [17]:
data_array = [TRANS(context[i],) for i in sli]

TypeError: TRANS() missing 2 required positional arguments: 'a' and 'arm_size'